特别感谢@[混沌](https://www.joinquant.net/user/c9f1650814bdb9f6a628ec2919f61bba)大佬

使用Python相关算法包一是减少了要编写以及调试的代码行数，二是包的运行速度要远快于之前的代码

不同版本的包使用方式和注意事项不同，需要查看函数说明

为了精细控制交叉验证每一折验证使用的训练集和测试集，可能不能直接使用交叉验证的函数版本：
* 类别属性的某个属性值出现较少，就需要通过抽样来确保每份数据中包含特定属性值的样本个数是等比例的
* 也需要访问每折数据来计算错误统计量，比如不想使用MSE二是MAE，因为它能更好地代表实际问题中的错误
* 需要对每折数据计算错误的情况使用线性回归来解决分类问题。例如误分率或AUC

In [1]:
#-*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LassoCV
import warnings
warnings.filterwarnings('ignore')

**1) 多变量回归：预测红酒口感**

之所以把红酒口感看成回归问题，是因为口感是有程度的、有序的， 而分类问题本质只有区别而无优劣次序。

具体是3~8分6种可能的类别，但是考虑到5比6差，又比4好，且实际的口感得分为3时，预测值为5要比预测值为4对累积错误的贡献更大，所以回归比分类更好体现

> **构建并测试模型以预测红酒的口感**

构建模型的第一步是通过样本外的性能来判断模型能否满足性能要求。

本例展示了用算法包sklearn.linear_model.LassoCV进行套索线性回归的10折交叉验证。

下面三个图为使用不同归一化版本的预测效果，对应如下：
1. X和Y都做归一

2. X做归一，Y不做归一

3. X和Y都不做归一

In [2]:
# 获取红酒数据集
df = pd.read_csv('../input/winequality/winequality-red.csv')
df.columns = ['非挥发性酸','挥发性酸','柠檬酸', '残留糖分', '氯化物', '游离二氧化硫','总二氧化硫', '密度', 
              'PH值', '亚硝酸盐', '酒精含量', '品质']

In [3]:
# 标准化 切分属性和标签
norm_df = (df - df.mean())/df.std()
xData = df.values[:,:-1]; yData = df.values[:,-1] 
xNormData = norm_df.values[:,:-1]; yNormData = norm_df.values[:,-1] 
m, n = xData.shape

In [4]:
#  标准化与否的选择
# xx = xData; 
xx = xNormData
#y = yData
y = yNormData

# 调用sklearn.linear_model中的LassoCV 
wineModel = LassoCV(cv=10).fit(xx, y)

# 显示结果
plt.figure()
plt.plot(wineModel.alphas_, wineModel.mse_path_, ':')
plt.plot(wineModel.alphas_, wineModel.mse_path_.mean(axis=-1),
         label='Average MSE Across Folds', linewidth=2)
plt.axvline(wineModel.alpha_, linestyle='--',
            label='CV Estimate of Best alpha')
plt.semilogx()
plt.legend()
ax = plt.gca()
ax.invert_xaxis()
plt.xlabel('alpha')
plt.ylabel('Mean Square Error')
plt.axis('tight')
plt.show()

#print out the value of alpha that minimizes the Cv-error
print("最小均方误差对应的alpha值  ",wineModel.alpha_)
print("最小均方误差：", min(wineModel.mse_path_.mean(axis=-1)))

In [5]:
#  标准化与否的选择
# xx = xData; 
xx = xNormData
y = yData
# y = yNormData

# 调用sklearn.linear_model中的LassoCV 
wineModel = LassoCV(cv=10).fit(xx, y)

plt.figure()
plt.plot(wineModel.alphas_, wineModel.mse_path_, ':')
plt.plot(wineModel.alphas_, wineModel.mse_path_.mean(axis=-1),
         label='Average MSE Across Folds', linewidth=2)
plt.axvline(wineModel.alpha_, linestyle='--',
            label='CV Estimate of Best alpha')
plt.semilogx()
plt.legend()
ax = plt.gca()
ax.invert_xaxis()
plt.xlabel('alpha')
plt.ylabel('Mean Square Error')
plt.axis('tight')
plt.show()

#print out the value of alpha that minimizes the Cv-error
print("最小均方误差对应的alpha值  ",wineModel.alpha_)
print("最小均方误差：", min(wineModel.mse_path_.mean(axis=-1)))

In [6]:
#  标准化与否的选择
xx = xData; 
# xx = xNormData
y = yData
# y = yNormData

# 调用sklearn.linear_model中的LassoCV 
wineModel = LassoCV(cv=10).fit(xx, y)

plt.figure()
plt.plot(wineModel.alphas_, wineModel.mse_path_, ':')
plt.plot(wineModel.alphas_, wineModel.mse_path_.mean(axis=-1),
         label='Average MSE Across Folds', linewidth=2)
plt.axvline(wineModel.alpha_, linestyle='--',
            label='CV Estimate of Best alpha')
plt.semilogx()
plt.legend()
ax = plt.gca()
ax.invert_xaxis()
plt.xlabel('alpha')
plt.ylabel('Mean Square Error')
plt.axis('tight')
plt.show()

#print out the value of alpha that minimizes the Cv-error
print("最小均方误差对应的alpha值  ",wineModel.alpha_)
print("最小均方误差：", min(wineModel.mse_path_.mean(axis=-1)))

对X、y是否做标准化的其它选择后(调整代码中xx,y的赋值后重新绘图)，我们会发现：

1） X标准化，y不做标准化： 虽然均方误差显著变小，但图形很类似没有本质区别，仅仅因为对标签进行标准化会失去与原始数据的关联而已。实际上我们可以将标准化后的标签转换会原来的标签尺度，并且这种转换的实现都是现成。所以对标签的标准化与否没有本质区别。

2） X、y都不做标准化： 虽然均方误差变化不大，但均方误差曲线有扇形的陡然下降区域，这是因为X特征尺度混乱导致，算法挑选了尺度大的变量进行预测，对应的系数很小。算法使用一个较差的变量进行若干次迭代，直到α变得足够小以引进一个更好的变量，这时错误才会陡然下降。

**本例建议**：应该对特征属性和标签值都做标准化。

> **部署前在整个数据集上进行训练**

本例展示了用算法包sklearn.linear_model.lasso_path进行套索线性回归路径分析。

所谓路径分析，就是一组回归分析，本例的一组是由alphas控制，每个alpha对应一个线性回归。

In [7]:
#  标准化与否的选择
# xx = xData; 
xx = xNormData
# y = yData
y = yNormData


alphas, coefs, _  = linear_model.lasso_path(xx, y,  return_models=False)

plt.plot(alphas,coefs.T)

plt.xlabel('alpha')
plt.ylabel('Coefficients')
plt.axis('tight')
plt.semilogx()
ax = plt.gca()
ax.invert_xaxis()
plt.show()

nattr, nalpha = coefs.shape

# 回归系数排序
nzList = []
for iAlpha in range(1,nalpha):
    coefList = coefs[: ,iAlpha]
    
    # 记录回归系数刚好变成非零的属性
    nzCoef = np.where(coefList!=0)[0]
    for q in nzCoef:
        if q not in nzList:
            nzList.append(q)

print("系数进入模型的次序所对应的属性排序 :",)
for idx in nzList:
    print(df.columns[idx],end=' ')
print("")

# 根据前面已获得的最佳\alpha，寻找此例中对应的索引
alphaStar = 0.013561387700964642
indexLTalphaStar = [index for index in range(100) if alphas[index] > alphaStar]
indexStar = max(indexLTalphaStar)

# 进而根据上面“对应的索引”，获得最佳回归系数值
coefStar = coefs[:,indexStar]
print("最佳回归系数：", coefStar)

# 回归系数给出了另外一组稍微不同的顺序
absCoef =  np.abs(coefStar)
idxs = np.argsort(-absCoef)

print("最优alpha的系数尺度所对应的属性排序 :",)
for idx in idxs:
    if absCoef[idx]!=0:
        print(df.columns[idx],end=' ')
print ("")

In [8]:
#  标准化与否的选择
xx = xData; 
# xx = xNormData
# y = yData
y = yNormData


alphas, coefs, _  = linear_model.lasso_path(xx, y,  return_models=False)

plt.plot(alphas,coefs.T)

plt.xlabel('alpha')
plt.ylabel('Coefficients')
plt.axis('tight')
plt.semilogx()
ax = plt.gca()
ax.invert_xaxis()
plt.show()

nattr, nalpha = coefs.shape

# 回归系数排序
nzList = []
for iAlpha in range(1,nalpha):
    coefList = coefs[: ,iAlpha]
    
    # 记录回归系数刚好变成非零的属性
    nzCoef = np.where(coefList!=0)[0]
    for q in nzCoef:
        if q not in nzList:
            nzList.append(q)

print("系数进入模型的次序所对应的属性排序 :",)
for idx in nzList:
    print(df.columns[idx], end=' ')
print("")

# 根据前面已获得的最佳\alpha，寻找此例中对应的索引
alphaStar = 0.013561387700964642
indexLTalphaStar = [index for index in range(100) if alphas[index] > alphaStar]
indexStar = max(indexLTalphaStar)

# 进而根据上面“对应的索引”，获得最佳回归系数值
coefStar = coefs[:,indexStar]
print("最佳回归系数：", coefStar)

# 回归系数给出了另外一组稍微不同的顺序
absCoef =  np.abs(coefStar)
idxs = np.argsort(-absCoef)

print("最优alpha的系数尺度所对应的属性排序 :",)
for idx in idxs:
    if absCoef[idx]!=0:
        print(df.columns[idx],end=' ')
print ("")

对于标准化属性， 上面两种属性的排序基本一致，但在不太重要的属性上有所差异。

对于非标准化属性，系数进入模型的次序几乎是混乱的，但最优alpha的系数尺度次序变化不大。几个早期进入解得系数相对于后续进入解得系数更接近于0，这种现象正好证明了系数进入模型的顺序与最佳解得系数尺度的顺序存在本质不同

> **基扩展：基于原始属性扩展新属性来改进性能**

本例尝试扩展了两个属性。

In [9]:
# 沿用前面已经获得的数据

# 扩展2个新属性
xExtData = np.concatenate((xData, np.zeros((m,2))), axis=1)
xExtData[:, n] = xData[:,-1]**2
xExtData[:, n+1] = xData[:,-1]*xData[:,1]

xNormExtData = (xExtData - xExtData.mean(axis=0))/np.std(xExtData,axis=0)

m, n = xData.shape

# 更新属性名
names = list(df.columns[idx])
names[-1] = "alco^2"
names.append("alco*volAcid")

#  标准化与否的选择
# xx = xExtData
xx = xNormExtData
# y = yData
y = yNormData

# 调用sklearn.linear_model中的LassoCV 
wineModel = LassoCV(cv=10).fit(xx, y)

# 显示结果
plt.figure()
plt.plot(wineModel.alphas_, wineModel.mse_path_, ':')
plt.plot(wineModel.alphas_, wineModel.mse_path_.mean(axis=-1),
         label='Average MSE Across Folds', linewidth=2)
plt.axvline(wineModel.alpha_, linestyle='--',
            label='CV Estimate of Best alpha')
plt.semilogx()
plt.legend()
ax = plt.gca()
ax.invert_xaxis()
plt.xlabel('alpha')
plt.ylabel('Mean Square Error')
plt.axis('tight')
plt.show()

#print out the value of alpha that minimizes the Cv-error
print("最小均方误差对应的alpha值  ",wineModel.alpha_)
print("最小均方误差：", min(wineModel.mse_path_.mean(axis=-1)))

代码中尝试添加了两个扩展属性，但似乎没有多大效果。 如果要获得更好的效果，似乎要做不少尝试。

**2) 二分类：检测水雷**

“水雷&岩石”只有区别而无本质优劣，所以是分类问题。本例使用scikit-learn算法包中的ElasticNet包。

> **构建并测试模型**

依然是先通过样本外的性能来判断模型能否满足性能要求。

本例计划用ElasticNet线性回归解决分类问题，但是10折交叉验证算法sklearn.linear_model.ElasticNetCV是基于均方误差的，而我们的分类问题的性能度量需要基于误分率或AUC的,显然不合适。 为此，我们不得不手工构建10折交叉验证循环。

在手工10折交叉验证循环中用算法sklearn.linear_model.enet_path进行elasticNet线性回归路径分析。

本例展示了度量分类器性能的两种方法: 误分率和AUC(ROC曲线下面积),分别对应后面的第一张图和第二张图。 第三张图则绘制基于最佳auc的ROC曲线。

AUC的优势在于：能获得最佳性能并且和应用场景无关。**但不能确保在一个特定的误分率上获得最优性能。**

一般而言， 误分率似乎是我们的更好选择。

In [10]:
from sklearn.linear_model import enet_path
from sklearn.metrics import roc_auc_score, roc_curve

df = pd.read_csv('../input/sonaralldata/sonar.all-data.csv', header=None, prefix='V')

# 分类标签数值化
df['V60'] = df.iloc[:,-1].apply(lambda v: 1.0 if v=='M' else 0.0)

# 切分属性和标签 然后 标准化 
xData = df.values[:,:-1]; yData = df.values[:,-1] 

xData = (xData - xData.mean(axis=0))/xData.std(axis=0)
yData = (yData - yData.mean())/yData.std()

m, n = xData.shape

# 手工构建10折交叉验证循环
nxval = 10
for ixval in range(nxval):
    
    # 第ixval折验证的训练集和测试集的切分
    idxTest = [i for i in range(m) if i%nxval == ixval%nxval]
    idxTrain = [i for i in range(m) if i%nxval != ixval%nxval]    
    xTest = xData[idxTest,:]; yTest = yData[idxTest]
    xTrain = xData[idxTrain,:]; yTrain = yData[idxTrain]

    # enet_path 就是 ElasticNet正规化路径
    # 参数：l1_ratio就是套索惩罚项的占比
    alphas, coefs, _ = enet_path(xTrain, yTrain,l1_ratio=0.8, fit_intercept=False, return_models=False)

    # 将所有额预测及其标签归集到一起
    if ixval == 0:
        pred = np.dot(xTest, coefs)
        yOut = yTest
    else:
        pred = np.concatenate((pred, np.dot(xTest, coefs)), axis = 0)
        yOut = np.concatenate((yOut, yTest), axis=0)  

# 计算误分率
misClassRate = 1.0*((pred>=0)^(np.repeat(yOut, pred.shape[1]).reshape(pred.shape)>=0)).sum(axis=0)/m

# 寻找最小误差
idxMin = misClassRate.argmin()
minError = misClassRate[idxMin]

plt.figure()
plt.plot(alphas[1:], misClassRate[1:], label='Misclassification Error Across Folds', linewidth=2)
plt.axvline(alphas[idxMin], linestyle='--', label='CV Estimate of Best alpha')
plt.legend()
plt.semilogx()
ax = plt.gca()
ax.invert_xaxis()
plt.xlabel('alpha')
plt.ylabel('Misclassification Error')
plt.axis('tight')
plt.show()

# 计算AUC
auc = []
for iPred in range(0, pred.shape[1]):
    predList = list(pred[:, iPred])
    aucCalc = roc_auc_score((yOut>0.0), predList)
    auc.append(aucCalc)

# 最大auc 及其对应的最小误分率
idxMax = np.argmax(auc)
minError = misClassRate[idxMin]

plt.figure()
plt.plot(alphas[1:], auc[1:], label='AUC Across Folds', linewidth=2)
plt.axvline(alphas[idxMax], linestyle='--', label='CV Estimate of Best alpha')
plt.legend()
plt.semilogx()
ax = plt.gca()
ax.invert_xaxis()
plt.xlabel('alpha')
plt.ylabel('Area Under the ROC Curve')
plt.axis('tight')
plt.show()


# 绘制最佳性能分类器的ROC曲线
fpr, tpr, thresh = roc_curve((yOut>0.0), list(pred[:, idxMax]))
ctClass = [i*0.01 for i in range(101)]
plt.plot(fpr, tpr, linewidth=2)
plt.plot(ctClass, ctClass, linestyle=':')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

print('最佳误分率 = ', misClassRate[idxMin])
print('最佳alpha(对应最佳误分率) = ', alphas[idxMin])
print('')
print('最佳AUC = ', auc[idxMax])
print('最佳alpha(对应最佳AUC)   =  ', alphas[idxMax])

print('')
print('不同阈值对应的混淆矩阵' )

# 正例数（标签为水雷）
P = len(yOut[yOut>0.0])

thr_idx = 20
print('')
print('阈值 =   ', thresh[thr_idx])
print('真正数(TP) = ', tpr[thr_idx]*P, '假负数(FN) = ', (1-tpr[thr_idx])*P)
print('假正数(FP) = ', fpr[thr_idx]*(m-P),'真负数(TN) = ', (1-fpr[thr_idx])*(m-P) )

thr_idx = 40
print('')
print('阈值 =   ', thresh[thr_idx])
print('真正数(TP) = ', tpr[thr_idx]*P, '假负数(FN) = ', (1-tpr[thr_idx])*P)
print('假正数(FP) = ', fpr[thr_idx]*(m-P),'真负数(TN) = ', (1-fpr[thr_idx])*(m-P) )

thr_idx = 60
print('')
print('阈值 =   ', thresh[thr_idx])
print('真正数(TP) = ', tpr[thr_idx]*P, '假负数(FN) = ', (1-tpr[thr_idx])*P)
print('假正数(FP) = ', fpr[thr_idx]*(m-P),'真负数(TN) = ', (1-fpr[thr_idx])*(m-P) )

阈值确定需要合理的代价设置，同时要尽量确保训练集的正例和负例的比例与实际情况相一致。

> **构建并部署分类器**

完成 α 的选择后， 现在应该在全部数据集上重新训练模型, 然后确定 α 在新的模型下所对应的回归系数。

新模型的路径分析和最佳回归系数，各自对应一组属性的排序，这正好是**特征工程**所需要的信息。

衡量变量重要性的一个指标是随着α减小，进入解得变量顺序。另一个指标是根据最优解的特征系数大小得到的排序。

In [11]:
# 沿用已有的数据

# 在整个数据集上进行路径分析
alphas, coefs, _ = enet_path(xData, yData,l1_ratio=0.8, fit_intercept=False, return_models=False)

plt.plot(alphas,coefs.T)
plt.xlabel('alpha')
plt.ylabel('Coefficients')
plt.axis('tight')
plt.semilogx()
ax = plt.gca()
ax.invert_xaxis()
plt.show()

nattr, nalpha = coefs.shape

# 回归系数排序
nzList = []
for iAlpha in range(1,nalpha):
    coefList = coefs[: ,iAlpha]
    
    # 记录回归系数刚好变成非零的属性
    nzCoef = np.where(coefList!=0)[0]
    for q in nzCoef:
        if q not in nzList:
            nzList.append(q)

print("系数进入模型的次序所对应的属性排序 :",)
for idx in nzList:
    print(df.columns[idx],end=' ')
print("")

# 根据前面已获得的最佳\alpha，寻找此例中对应的索引
alphaStar = 0.020334883589342503
indexLTalphaStar = [index for index in range(100) if alphas[index] > alphaStar]
indexStar = max(indexLTalphaStar)
print(indexStar)

# 进而根据上面“对应的索引”，获得最佳回归系数值
coefStar = coefs[:,indexStar]
print("最佳回归系数：", coefStar)

# 回归系数给出了另外一组稍微不同的顺序
absCoef =  np.abs(coefStar)
idxs = np.argsort(-absCoef)

print("最优alpha的系数尺度所对应的属性排序 :",)
for idx in idxs:
    if absCoef[idx]!=0:
        print(df.columns[idx],end=' ')
print("")

**3) 逻辑回归与惩罚逻辑回归**

另一种使用惩罚线性回归模型进行分类的方法是使用惩罚逻辑回归。

IRLS方法也叫Newton-Raphson，我们可以用这个方法求解惩罚逻辑回归问题。

In [12]:
def S(z,gamma):
    if gamma >= np.abs(z):
        return 0.0
    if z > 0.0:
        return z - gamma
    else:
        return z + gamma

def glmnetIRLS(xx, y, beta, lam, alp):
    '''
        Glmnet和IRLS两个算法结合在一起，用于解决惩罚逻辑回归问题. 这是一个单步迭代公式
    参数：
        xx:  属性矩阵X
        y:   标签向量
        beta: 回归系数向量(更新后返回)
        lam:  \lambda
        alp:  \alpha
    '''

    m,n = xx.shape
       
    eta = np.dot(xx, beta) # = X beta
    eta[eta<-100] = -100
    
    mu = 1.0/(1.0+np.exp(-eta))
    
    w = mu * (1.0 - mu)
    
    cond1 = (np.abs(mu)<1e-5)
    cond2 = (np.abs(1.0-mu)<1e-5)
    mu[cond1] = 0.0
    mu[cond2] = 1.0
    w[cond1|cond2] = 1e-5
    
    ww = np.diag(w)
    
    z = (y-mu)/w + eta # = eta + W^{-1} (y - mu)
    
    for j in range(n):
        r = z - np.dot(xx,beta) # = z - X beta
        
        sumWxr = (w*xx[:,j]*r).sum()  # = (X^T W r)[j]
        sumWxx =  (w*xx[:,j]*xx[:,j]).sum() # = (X^T W X)[j,j]
        
        beta[j] = beta[j] + sumWxr / sumWxx
        
        if j > 0:
            avgWxx = sumWxx / m
            beta[j] = S(beta[j]*avgWxx, lam * alp) / (avgWxx + lam * (1.0 - alp))
    
    return beta

# 分类标签数值化
#df['V60'] = df.iloc[:,-1].apply(lambda v: 1.0 if v=='M' else 0.0)

# 切分属性和标签 然后 标准化 
xData = df.values[:,:-1]; yData = df.values[:,-1] 

# 只对X正规化, y只计算均值和标准差
xData = (xData - xData.mean(axis=0))/xData.std(axis=0)
yMean = yData.mean();  yStd = yData.std()

# 选择alpha参数
alpha = 0.8  # ElasticNet回归

# 确定lambda初始值：导致所有beta值都为零的lambda值（最简模型）
lam = (np.abs(yData.dot(xData))/xData.shape[0]).max()/alpha

# 扩展X成[1,X]
xData = np.concatenate((np.ones((xData.shape[0],1)), xData), axis=1)
nrow, ncol = xData.shape

# beta初始化
beta = np.zeros(ncol)
beta[0] = np.log(yMean/(1-yMean))

# 记录历史beta
betaMat = []
betaMat.append(list(beta[1:]))
beta0Mat = []
beta0Mat.append(beta[0])

# 迭代步数
nSteps = 100

# lambda缩减乘子：Fredman建议每步迭代后都要稍微减小lambda
lamMult = 0.93 # Fredman建议: lamMult^nSteps = 0.001

# 记录属性回归系数变成非零先后次序
nzList = []

# 开始进行lam迭代计算
for iStep in range(nSteps):
    lam = lam * lamMult
    
    deltaBeta = 100.0
    eps = 0.01
    iterStep = 0
    
    # 开始Glmnet算法迭代
    while deltaBeta > eps:
        iterStep += 1
        if iterStep > 100: #100:
            break
        
        # 上一步beta
        _beta = beta.copy()

        # IRL算法递归公式
        beta = glmnetIRLS(xData, yData, beta, lam, alpha)
        
        # 计算精度
        deltaBeta = np.abs(beta[1:]-_beta[1:]).sum() / np.abs(beta[1:]).sum()
    
    # 记录beta历史
    betaMat.append(list(beta[1:]))
    beta0Mat.append(beta[0])

    # 记录回归系数刚好变成非零的属性
    nzBeta = np.where(beta[1:]!=0)[0]
    for q in nzBeta:
        if (q in nzList) == False:
            nzList.append(q)

# 打印属性重要性排序
for idx in nzList:
    print(df.columns[idx],end=' ')
print("")

# lambda-误差曲线
plt.plot(betaMat)
plt.xlabel("Step Taken")
plt.ylabel("Coefficient Values")
plt.show()

**4) 多类别分类**

本例采用方法的核心就是：将分类标签映射到10向量（一个1多个0，1的位置就是标签集索引），然后每个标签构建一个模型。 采用10折交叉验证的方法，计算惩罚步数所对应的误分率，并绘制成图。

在手工10折交叉验证循环中用算法sklearn.linear_model.enet_path进行elasticNet线性回归路径分析。

In [19]:
df = pd.read_csv('../input/glassdata/glass.data.csv', header=None, prefix="V")
df.columns = ['Id','RI','Na', 'Mg', 'Al', 'Si','K', 'Ca', 'Ba', 'Fe', 'Type']
df = df.set_index("Id")

# 切分属性和标签
xData = df.values[:,:-1]; yLabel = df.values[:,-1] 

# 标签值向量化(一对所有)
labelList = list(set(yLabel))
labelList.sort()
nlabels = len(labelList)
def mapFunc(label):
    idx = labelList.index(label)
    row = [0]*nlabels
    row[idx] = 1
    return row
yData = np.array([mapFunc(label) for label in yLabel])

# 标准化
xData = (xData - xData.mean(axis=0))/xData.std(axis=0)
yMean = yData.mean(axis=0); yStd = yData.std(axis=0)
yData = (yData - yMean)/yStd

# 数据规模
m, n = xData.shape

# 手工构建n折交叉验证循环
nxval = 10
nAlphas= 100
misClass = [0.0] * nAlphas
for ixval in range(nxval):   
    # 第ixval折验证的训练集和测试集的切分
    idxTest = [i for i in range(m) if i%nxval == ixval%nxval]
    idxTrain = [i for i in range(m) if i%nxval != ixval%nxval]    
    xTest = xData[idxTest,:]; yTest = yData[idxTest,:]
    xTrain = xData[idxTrain,:]; yTrain = yData[idxTrain,:]
    labelTest = yLabel[idxTest]    

    # 为yTrain的每列建立模型
    # enet_path 就是 ElasticNet正规化路径
    # 参数：l1_ratio就是套索惩罚项的占比
    models = [enet_path(xTrain, yTrain[:,k] ,l1_ratio=1.0, fit_intercept=False, 
                        eps=0.5e-3, n_alphas=nAlphas , return_models=False) 
              for k in range(nlabels)]
    
    lenTest = m - len(yTrain)
    for iStep in range(1,nAlphas):
        # 组合所有模型的预测
        allPredictions = []
        for iModel in range(nlabels):
            # 模型预测
            _, coefs, _ = models[iModel]
            predTemp = np.dot(xTest, coefs[:,iStep])
            
            # 去标准化后比较
            predUnNorm = predTemp*yStd[iModel]+yMean[iModel]
            allPredictions.append(list(predUnNorm))
        allPredictions = np.array(allPredictions)

        # 找出最大的预测和误差        
        predictions = []
        for i in range(lenTest):
            listOfPredictions = allPredictions[:,i]
            idxMax = listOfPredictions.argmax()
            if labelList[idxMax] != labelTest[i]:
                misClass[iStep] += 1.0

misClassPlot = [misClass[i]/m for i in range(1, nAlphas)]

plt.plot(misClassPlot)

plt.xlabel("Penalty Parameter Steps")
plt.ylabel(("Misclassification Error Rate"))
plt.show()